# GLIDE Data with pystac-monty

This notebook demonstrates how to use pystac-monty to process GLIDE disaster data and visualize it using interactive maps. We'll:

1. Download GLIDE data from the GLIDE API
2. Convert the data to STAC items using pystac-monty
3. Display events on an interactive map
4. Explore the Monty STAC model and its metadata

Let's begin by importing the necessary libraries.

In [35]:
%load_ext autoreload
%autoreload 2

# Basic libraries
import tempfile
import json

# Visualization libraries
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import requests

# STAC and pystac-monty
import shapely
from folium.plugins import MarkerCluster
from IPython.display import clear_output, display

from pystac_monty.extension import MontyExtension
from pystac_monty.geocoding import WorldAdministrativeBoundariesGeocoder
from pystac_monty.sources.glide import GlideDataSource, GlideTransformer


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Download and Process GLIDE Data

First, let's download GLIDE data for a specific country and event type, and initialize the data source and transformer.

In [29]:
# Define GLIDE API parameters
country_code = "*"  # Spain
event_type = "*"     # Flood
from_year = ""
to_year = "2025"

# Construct the GLIDE API URL
glide_url = f"https://www.glidenumber.net/glide/jsonglideset.jsp?level1={country_code}&fromyear={from_year}&toyear={to_year}&events={event_type}"

print(f"Downloading GLIDE data from {glide_url}...")
response = requests.get(glide_url)
glide_data = response.text
print(f"Downloaded {len(glide_data)} bytes")

# Initialize the data source and transformer
data_source = GlideDataSource(glide_url, glide_data)
transformer = GlideTransformer(data_source)

# Initialize the geocoder
geocoder = WorldAdministrativeBoundariesGeocoder("../tests/data-files/world-administrative-boundaries.fgb")

Downloaded 4991153 bytes


Let's examine the raw GLIDE data to understand its structure:

In [30]:
# Parse and display the GLIDE data
glide_json = json.loads(glide_data)
print(f"Number of GLIDE events: {len(glide_json.get('glideset', []))}")

# Display the first event if available
if glide_json.get('glideset') and len(glide_json.get('glideset')) > 0:
    print("\nSample GLIDE event:")
    print(json.dumps(glide_json.get('glideset')[0], indent=2))

Number of GLIDE events: 8384

Sample GLIDE event:
{
  "comments": "On 13 March, a major oil pipeline ruptured inland Ecuador, contaminating several rivers and outpouring into coastal areas.\r\nMore than 300,000 people are affected, primarily due to the suspension of potable water in the area. The lack of safe access to water increases the risk of disease and heightens vulnerabilities and needs in affected communities that are already highly marginalized.",
  "year": 2025,
  "docid": 23482,
  "latitude": -1.831239,
  "homeless": 0,
  "source": "OCHA",
  "idsource": "",
  "killed": 0,
  "affected": 0,
  "duration": 0,
  "number": "2025-000037",
  "injured": 0,
  "month": 3,
  "geocode": "ECU",
  "location": "",
  "magnitude": "",
  "time": "",
  "id": "",
  "event": "AC",
  "day": 13,
  "status": "A",
  "longitude": -78.183406
}


## 2. Create STAC Items from GLIDE Data

Now, let's transform the GLIDE data into STAC items.

In [43]:
# Create STAC items
print("Creating STAC items from GLIDE data...")
all_stac_items = transformer.make_items()
print(f"Created {len(all_stac_items)} STAC items")

Creating STAC items from GLIDE data...
Error creating date from {'year': 201, 'month': 4, 'day': 10}: Invalid isoformat string: '201-04-10T00:00:00.000+00:00'
Error creating date from {'year': 201, 'month': 4, 'day': 10}: Invalid isoformat string: '201-04-10T00:00:00.000+00:00'
Created 16768 STAC items


In [44]:
# Separate the STAC items by role
event_items = []
hazard_items = []
impact_items = []

for item in all_stac_items:
    roles = item.properties.get("roles", [])
    if "event" in roles:
        event_items.append(item)
    elif "hazard" in roles:
        hazard_items.append(item)
    elif "impact" in roles:
        impact_items.append(item)

print(f"Events: {len(event_items)}, Hazards: {len(hazard_items)}, Impacts: {len(impact_items)}")

Events: 8384, Hazards: 8384, Impacts: 0


Let's examine one of the STAC event items to understand its structure:

In [45]:
# Display the first event item if available
if event_items:
    print("Sample STAC event item:")
    print(json.dumps(event_items[0].to_dict(), indent=2))

Sample STAC event item:
{
  "type": "Feature",
  "stac_version": "1.1.0",
  "stac_extensions": [
    "https://ifrcgo.github.io/monty/v0.1.0/schema.json"
  ],
  "id": "glide-event-AC-2025-000037-ECU",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -78.183406,
      -1.831239
    ]
  },
  "bbox": [
    -78.183406,
    -1.831239,
    -78.183406,
    -1.831239
  ],
  "properties": {
    "title": "",
    "description": "On 13 March, a major oil pipeline ruptured inland Ecuador, contaminating several rivers and outpouring into coastal areas.\r\nMore than 300,000 people are affected, primarily due to the suspension of potable water in the area. The lack of safe access to water increases the risk of disease and heightens vulnerabilities and needs in affected communities that are already highly marginalized.",
    "magnitude": "",
    "source": "OCHA",
    "docid": 23482,
    "status": "A",
    "keywords": [
      "AC",
      "glide",
      "ECU"
    ],
    "roles": [
      "so

## 3. Add Items to STAC Collections

Now, let's add the STAC items to their respective collections in a STAC API.

In [47]:
# Import STAC helper functions
import sys
sys.path.append('.')
from stac_helpers import (
    add_items_to_collection,
    check_collection_exists,
    check_stac_api_availability,
    create_collection_fallback,
    create_collection_from_file,
    delete_collection,
)

In [48]:
# Define the STAC API endpoint
# Replace with your actual STAC API endpoint
stac_api_url = "https://montandon-eoapi-stage.ifrc.org/stac"

# Define the collection IDs for each type of item
# These match the predefined collections in monty-stac-extension/examples
event_collection_id = "glide-events"
hazard_collection_id = "glide-hazards"

# Define paths to the predefined collection definitions
event_collection_path = "../monty-stac-extension/examples/glide-events/glide-events.json"
hazard_collection_path = "../monty-stac-extension/examples/glide-hazards/glide-hazards.json"

# Check if the STAC API is available
api_available = check_stac_api_availability(stac_api_url)

STAC API is available at https://montandon-eoapi-stage.ifrc.org/stac


In [49]:
# Check if the collections exist and create them if they don't
if api_available:
    # Check and create event collection if needed
    delete_collection(stac_api_url, event_collection_id)
    event_collection_exists = check_collection_exists(stac_api_url, event_collection_id)
    if not event_collection_exists:
        print(f"\nAttempting to create collection '{event_collection_id}'...")
        event_collection_created = create_collection_from_file(stac_api_url, event_collection_path)
        if not event_collection_created:
            print("Trying fallback method to create event collection...")
            event_collection_created = create_collection_fallback(
                stac_api_url, 
                event_collection_id, 
                "GLIDE events processed with pystac-monty",
                ["event", "source"]
            )
        event_collection_exists = event_collection_created
    
    # Check and create hazard collection if needed
    delete_collection(stac_api_url, hazard_collection_id)
    hazard_collection_exists = check_collection_exists(stac_api_url, hazard_collection_id)
    if not hazard_collection_exists:
        print(f"\nAttempting to create collection '{hazard_collection_id}'...")
        hazard_collection_created = create_collection_from_file(stac_api_url, hazard_collection_path)
        if not hazard_collection_created:
            print("Trying fallback method to create hazard collection...")
            hazard_collection_created = create_collection_fallback(
                stac_api_url, 
                hazard_collection_id, 
                "GLIDE hazards processed with pystac-monty",
                ["hazard", "source"]
            )
        hazard_collection_exists = hazard_collection_created
    
    if not (event_collection_exists and hazard_collection_exists):
        print("\nWarning: One or more collections could not be created in the STAC API.")
        print("Some items may not be added to the STAC API.")
else:
    print("STAC API is not available. Skipping collection checks and creation.")

Collection 'glide-events' exists in the STAC API
Collection 'glide-events' deleted successfully
Collection 'glide-events' does not exist in the STAC API

Attempting to create collection 'glide-events'...
Collection 'glide-events' created successfully
Collection 'glide-hazards' exists in the STAC API
Collection 'glide-hazards' deleted successfully
Collection 'glide-hazards' does not exist in the STAC API

Attempting to create collection 'glide-hazards'...
Collection 'glide-hazards' created successfully


In [ ]:
# Add the items to their respective collections if the API is available
if api_available:
    if event_collection_exists:
        print("Adding event items to the collection...")
        event_success, event_failed = add_items_to_collection(stac_api_url, event_collection_id, event_items, overwrite=True)
    else:
        print("Skipping adding event items because the collection doesn't exist")
        event_success, event_failed = 0, len(event_items)
    
    if hazard_collection_exists:
        print("\nAdding hazard items to the collection...")
        hazard_success, hazard_failed = add_items_to_collection(stac_api_url, hazard_collection_id, hazard_items)
    else:
        print("Skipping adding hazard items because the collection doesn't exist")
        hazard_success, hazard_failed = 0, len(hazard_items)
    
    total_success = event_success + hazard_success
    total_failed = event_failed + hazard_failed
    
    print(f"\nSummary: Added {total_success} items successfully, {total_failed} items failed")
else:
    print("Skipping adding items to collections because the API is not available")

Adding event items to the collection...
Processing batch 1 of 9 (1000 items)
Processing batch 2 of 9 (1000 items)
Processing batch 3 of 9 (1000 items)
Processing batch 4 of 9 (1000 items)
Processing batch 5 of 9 (1000 items)
Processing batch 6 of 9 (1000 items)
Processing batch 7 of 9 (1000 items)
Processing batch 8 of 9 (1000 items)
Processing batch 9 of 9 (384 items)
Added 8384 items successfully, 0 items failed

Adding hazard items to the collection...
Processing batch 1 of 9 (1000 items)


## 4. Visualize GLIDE Events on a Map

Let's create an interactive map to visualize the GLIDE events.

In [ ]:
# Create a map centered on the first event's location or a default location
if event_items:
    # Get the coordinates of the first event
    first_event = event_items[0]
    coords = first_event.geometry["coordinates"]
    center = [coords[1], coords[0]]  # [lat, lon]
else:
    # Default center (Madrid, Spain)
    center = [40.4168, -3.7038]

# Create the map
m = folium.Map(location=center, zoom_start=6)

# Add a marker cluster for the events
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each event
for item in event_items:
    # Get the coordinates
    coords = item.geometry["coordinates"]
    lat, lon = coords[1], coords[0]
    
    # Get event properties
    title = item.properties.get("title", "No title")
    description = item.properties.get("description", "No description")
    event_type = item.id.split("-")[2]  # Extract event type from ID
    date = item.datetime.strftime("%Y-%m-%d")
    
    # Create popup content
    popup_content = f"<b>ID:</b> {item.id}<br>"
    popup_content += f"<b>Type:</b> {event_type}<br>"
    popup_content += f"<b>Date:</b> {date}<br>"
    popup_content += f"<b>Title:</b> {title}<br>"
    popup_content += f"<b>Description:</b> {description[:200]}...<br>"
    
    # Add marker to the cluster
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(marker_cluster)

# Display the map
m

## 5. Explore GLIDE Data in Detail

Let's create a widget to explore the GLIDE events in more detail.

In [ ]:
# Create a dropdown to select an event
event_options = [(f"{item.id} - {item.datetime.strftime('%Y-%m-%d')}", i) for i, item in enumerate(event_items)]

if event_options:
    event_dropdown = widgets.Dropdown(
        options=event_options,
        description='Select Event:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )

    # Function to display event details
    def display_event_details(change):
        clear_output(wait=True)
        display(event_dropdown)
        
        selected_index = change['new']
        selected_event = event_items[selected_index]
        
        # Display event details
        print(f"\n=== Event Details: {selected_event.id} ===\n")
        print(f"Date: {selected_event.datetime.strftime('%Y-%m-%d')}")
        print(f"Location: {selected_event.geometry['coordinates'][1]}, {selected_event.geometry['coordinates'][0]}")
        print(f"Title: {selected_event.properties.get('title', 'No title')}")
        print(f"Description: {selected_event.properties.get('description', 'No description')}")
        print(f"Magnitude: {selected_event.properties.get('magnitude', 'Not specified')}")
        print(f"Source: {selected_event.properties.get('source', 'Not specified')}")
        print(f"Status: {selected_event.properties.get('status', 'Not specified')}")
        
        # Display Monty extension properties
        print("\n=== Monty Extension Properties ===\n")
        monty_props = {k: v for k, v in selected_event.properties.items() if k.startswith('monty:')}
        for key, value in monty_props.items():
            print(f"{key}: {value}")
        
        # Find corresponding hazard item
        hazard_id = selected_event.id.replace("glide-event-", "glide-hazard-")
        matching_hazards = [h for h in hazard_items if h.id == hazard_id]
        
        if matching_hazards:
            hazard_item = matching_hazards[0]
            print("\n=== Corresponding Hazard ===\n")
            print(f"Hazard ID: {hazard_item.id}")
            
            # Display hazard details if available
            hazard_detail = hazard_item.properties.get("monty:hazard_detail", {})
            if hazard_detail:
                print(f"Cluster: {hazard_detail.get('cluster', 'Not specified')}")
                print(f"Severity: {hazard_detail.get('severity_value', 'Not specified')} {hazard_detail.get('severity_unit', '')}")
                print(f"Estimate Type: {hazard_detail.get('estimate_type', 'Not specified')}")
        
        # Create a small map centered on the event
        coords = selected_event.geometry["coordinates"]
        event_map = folium.Map(location=[coords[1], coords[0]], zoom_start=8)
        folium.Marker(
            location=[coords[1], coords[0]],
            popup=selected_event.id,
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(event_map)
        
        display(event_map)

    # Register the callback
    event_dropdown.observe(display_event_details, names='value')
    
    # Display the dropdown
    display(event_dropdown)
    
    # Display the first event by default
    if event_options:
        display_event_details({'new': event_options[0][1]})
else:
    print("No events available to display.")

## 6. Query Different GLIDE Data

You can modify the parameters below to query different GLIDE data.

In [ ]:
# Function to query GLIDE data with different parameters
def query_glide_data(country_code, event_type, from_year, to_year):
    # Construct the GLIDE API URL
    glide_url = f"https://www.glidenumber.net/glide/jsonglideset.jsp?level1={country_code}&fromyear={from_year}&toyear={to_year}&events={event_type}"
    
    print(f"Downloading GLIDE data from {glide_url}...")
    response = requests.get(glide_url)
    glide_data = response.text
    print(f"Downloaded {len(glide_data)} bytes")
    
    # Parse the data
    glide_json = json.loads(glide_data)
    events = glide_json.get('glideset', [])
    print(f"Number of GLIDE events: {len(events)}")
    
    # Display the events in a table
    if events:
        # Extract relevant fields for the table
        table_data = []
        for event in events:
            table_data.append({
                'GLIDE Number': event.get('docid', ''),
                'Event Type': event.get('event', ''),
                'Country': event.get('geocode', ''),
                'Date': f"{event.get('year', '')}-{event.get('month', '')}-{event.get('day', '')}",
                'Title': event.get('title', ''),
                'Magnitude': event.get('magnitude', ''),
                'Status': event.get('status', '')
            })
        
        # Create and display the DataFrame
        df = pd.DataFrame(table_data)
        display(df)
    else:
        print("No events found for the specified parameters.")

# Create widgets for the query parameters
country_widget = widgets.Text(
    value='CHN',
    description='Country Code:',
    style={'description_width': 'initial'}
)

event_widget = widgets.Text(
    value='EQ',
    description='Event Type:',
    style={'description_width': 'initial'}
)

from_year_widget = widgets.Text(
    value='2008',
    description='From Year:',
    style={'description_width': 'initial'}
)

to_year_widget = widgets.Text(
    value='2008',
    description='To Year:',
    style={'description_width': 'initial'}
)

query_button = widgets.Button(
    description='Query GLIDE Data',
    button_style='primary',
    tooltip='Click to query GLIDE data with the specified parameters'
)

output = widgets.Output()

def on_query_button_clicked(b):
    with output:
        clear_output()
        query_glide_data(
            country_widget.value,
            event_widget.value,
            from_year_widget.value,
            to_year_widget.value
        )

query_button.on_click(on_query_button_clicked)

# Display the widgets
display(widgets.VBox([
    widgets.HBox([country_widget, event_widget]),
    widgets.HBox([from_year_widget, to_year_widget]),
    query_button,
    output
]))

# Display help information
print("Event Type Codes:")
print("EQ: Earthquake, FL: Flood, TC: Tropical Cyclone, DR: Drought, WF: Wildfire")
print("VO: Volcano, TS: Tsunami, CW: Cold Wave, EP: Epidemic, LS: Landslide")
print("\nCountry Codes: Use ISO 3-letter country codes (e.g., USA, JPN, CHN, ESP)")